In [1]:
import gym
import numpy as np
import torch

from environments import *

from stable_baselines3 import SAC, HerReplayBuffer
from stable_baselines3.common.cmd_util import make_vec_env

c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\torch\utils\tensorboard\__init__.py:2: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion
c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\stable_baselines3\common\cmd_util.py:5: FutureWarning: Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.
  warnings.warn(


In [2]:
env = make_vec_env("G1GoalDist-v1", 16)

c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
model = SAC(
    "MlpPolicy", 
    env, 
    verbose=1,
    policy_kwargs=dict(
        net_arch=[512, 512, 512, 512], 
        activation_fn=torch.nn.ELU,
        n_critics=2,
    ),
    batch_size=2048,
    learning_rate=1e-4,
    # learning_starts=1000,
    tensorboard_log="./logs/sac/"
)

Using cuda device


In [4]:
import copy

def soft_update(task_model, meta_model, tau=0.2):
    for target_param, local_param in zip(meta_model.parameters(), task_model.parameters()):
        target_param.data.copy_(target_param.data + tau * (local_param.data-target_param.data))

def reptile(n_reptile_iterations=100000, n_sac_iterations=50000):
    
    meta_policy = copy.copy(model.policy)
    tasks = model.env.env_method('sample_tasks', (n_reptile_iterations), indices=0)[0]

    for task in tasks:

        print("Task: ", task)
                
        model.env.env_method('reset_task', (task))
        model.policy = copy.copy(meta_policy)
        # model.replay_buffer.reset()

        model.learn(
            total_timesteps=n_sac_iterations, 
            log_interval=100,
            tb_log_name='reptile',
            reset_num_timesteps=False,
        )

        soft_update(model.policy, meta_policy, 0.2)


In [5]:
reptile()

Task:  {'direction': 0.4378768905570979, 'orienation': 0}
Logging to ./logs/sac/reptile_0
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 214       |
|    ep_rew_mean     | -1.67e+03 |
| time/              |           |
|    episodes        | 100       |
|    fps             | 366       |
|    time_elapsed    | 65        |
|    total_timesteps | 23920     |
| train/             |           |
|    actor_loss      | 27.7      |
|    critic_loss     | 19.2      |
|    ent_coef        | 0.866     |
|    ent_coef_loss   | -2.43     |
|    learning_rate   | 0.0001    |
|    n_updates       | 1488      |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 169       |
|    ep_rew_mean     | -1.48e+03 |
| time/              |           |
|    episodes        | 200       |
|    fps             | 364       |
|    time_elapsed    | 108       |
|    total_timesteps | 39632     |
